<h1>Streaming Data Processing: Streaming Data Pipelines</h1>

<h3>Este documento serve como complemento ao exercício que pode ser encontrado em https://googlecloud.qwiklabs.com/labs/2187/edit</h3>
<h2>Tarefa 1: Preparação</h2>
<h3>Balde (intervalo) no Cloud Storage</h3>

Execute o seguinte código para ter certeza que existe um balde (intervalo) no Cloud Storage com o nome do seu projeto (clique na célula abaixo e em "run" -> "run selected cells" - ou tecle Shift + Enter - ou clique no icone "►" - para executar o código):

In [ ]:
!gsutil mb gs://$(gcloud config get-value project)

<br/>O balde deve ser criado ou você deve ver um erro "bucket already exists" se ele ja existir.

<h3>Dataset no BigQuery</h3>
Execute o código abaixo para criar um dataset que será utilizado neste exercício.

In [ ]:
!bq mk --dataset demos

<h3>Tabela no BigQuery</h3>
Execute o código abaixo para criar uma tabela que será utilizada para gravar as mensagens recebidas do Pub/Sub.

In [ ]:
!bq mk \
--table \
$(gcloud config get-value project):demos.taxis \
ride_id:STRING,point_idx:INTEGER,latitude:FLOAT,longitude:FLOAT,timestamp:STRING,meter_reading:FLOAT,meter_increment:FLOAT,ride_status:STRING,passenger_count:INTEGER

<h3>Ativar API do Dataflow se ela já não estiver ativada</h3>
Execute o código abaixo para ativar a API do Dataflow. O comando não retorna linha nenhuma.

In [ ]:
!gcloud services enable dataflow

<h3>Clone (usando o CLOUD SHELL) do Github o repositório com o fluxo do Dataflow</h3>
Abra o cloud shell. Copie e execute o comando a seguir para clonar o repositório que contém dados em tempo real de táxis

<h3>Criar tópico local no Pub/Sub</h3>
O código neste exercício (retirado de um Codelab do Google) lê mensagens do tópico público em tempo real e re-envia essas mensagens para outro tópico do Pub/Sub. Usaremos esse código para copiar as mensagens para um tópico local e deste tópico local copiaremos para o BigQuery.<br/>
<br/>
Execute o comando a seguir para criar o tópico local com o nome "visualizer" (necessário para o código).

In [ ]:
!gcloud pubsub topics create visualizer

<h2>Tarefa 2: Examine o tópico em tempo real público do Pub/Sub</h2>
O Google mantém o tópico projects/pubsub-public-data/topics/taxirides-realtime com dados em tempo real de taxis para uso em exercícios. Execute o código abaixo para ler a quantidade de assinaturas nesse tópico:

In [ ]:
!echo assinaturas:
!expr $(gcloud pubsub topics list-subscriptions projects/pubsub-public-data/topics/taxirides-realtime | wc -l) / 2
!echo exemplos:
!gcloud pubsub topics list-subscriptions projects/pubsub-public-data/topics/taxirides-realtime --limit 10

<h2>Tarefa 3: Execute fluxo de dados em tempo real no Dataflow</h2>
<b>1 - Analise o arquivo AllRides.java</b><br/>
<br/>
O que faz o script?<br/>
<br/>
Você pode querer analisar esse arquivo durante a tarefa, então, para um acesso fácil a ele, abra uma nova aba no browser e cole o seguinte endereço: https://raw.githubusercontent.com/googlecodelabs/cloud-dataflow-nyc-taxi-tycoon/master/dataflow/src/main/java/com/google/codelabs/dataflow/AllRides.java
<br/><br/>
Se quiser analisar o arquivo neste notebook execute o comando abaixo.

In [ ]:
!git clone https://github.com/googlecodelabs/cloud-dataflow-nyc-taxi-tycoon
  
!cat ~/cloud-dataflow-nyc-taxi-tycoon/dataflow/src/main/java/com/google/codelabs/dataflow/AllRides.java

<br/><b>2 - Execute (no CLOUD SHELL) o fluxo de dados</b><br/>
Copie e execute os seguintes comandos para criar o fluxo de dados. Como se trata de um fluxo desenvolvido em java, é preciso compilar o código utilizando o Maven (mvn).

Atenção: escrever no Pub/Sub a 2,000 events por segundo ≃ $0.05/minuto.

<h2>Tarefa 4: Analise o fluxo de dados</h2>
<br/><b>1 - Abra o console do Dataflow</b><br/>
No console web do GCP, abra a lista de jobs do Dataflow. Você deve ver listado um job com nome iniciado por "allrides-". Clique no job para ver o gráfico de execução.
<br/><br/>
Este fluxo lê do tópico público do Pub Sub e salva os resultados num outro tópico do Pub Sub criado localmente no seu projeto.<br/><br/>
Aguarde o tempo de inicialização do gráfico. Pode demorar alguns minutos até o Dataflow começar a receber as mensagens em tempo real e re-enviar para o nosso tópico local.

<h2>Tarefa 5: Copie os dados para o BigQuery</h2>
<br/><b>1 - Abra o console do Pub/Sub</b><br/>
No console web do GCP, abra a lista de tópicos do Pub/Sub. Você deve ver listado o tópico visualizer criado anteriormente.
<br/><br/>
<br/><b>2 - Confira para ver se o tópico está recebendo mensagens</b><br/>
• No console web do GCP, abra a lista de ASSINATURAS (subscriptions) do Pub/Sub. Clique em "criar assinatura" (create subscription).<br/>
• Preencha o Id da assinatura como "taxis-teste"<br/>
• Selecione o tópico "visualizer" no campo "tópico"<br/>
• Escolha "pull" como tipo de entrega (delivery type)<br/>
• Não altere mais nenhum parametro e clique em "criar"<br/>
• Retorne para o console de Tópicos, e clique no tópico visualizer para ver os detalhes<br/>
• Na barra superior clique "ver mensagens"<br/>
• Escolha a assinatura de teste que acabou de criar (taxis-teste) e clique em "pull"<br/>
<br/><br/>
Pode ser necessário clicar mais de uma vez. As mensagens só vão começar a chegar depois que o fluxo do Dataflow estiver já em pleno funcionamento.


<br/><b>3 - Use um modelo do Dataflow para copiar o tópico para o BigQuery</b><br/>
No próprio painel de tópico, clique no botão "Exportar para o BigQuery" (export to bigquery).
<br/><br/>
• Aceite o nome e região sugeridos para o fluxo do Dataflow que será criado.<br/>
• Selecione o modelo "Tópico do Pub/Sub para o BigQuery" (Pub/Sub Topic to BigQuery). Cuidado para <b>NÃO</b> escolher o modelo "assinatura do Pub/Sub..."<br/>
• Execute o código a seguir para gerar o nome do tópico:<br/>

In [2]:
!echo projects/$(gcloud config get-value project)/topics/visualizer

projects/active-bolt-265320/topics/visualizer


• Copie e cole o nome completo de tópico acima para o campo "tópico pub/sub de entrada" (input pub/sub topic)<br/>
• Execute o código a seguir para gerar o nome da tabela:<br/>

In [3]:
!echo $(gcloud config get-value project):demos.taxis

active-bolt-265320:demos.taxis


• Copie e cole o nome completo da tabela acima para o campo "tabela de saida do BigQuery" (BigQuery output table)<br/>
• Execute o código a seguir para gerar o nome da area temporaria que o modelo necessita:<br/>

In [5]:
!echo gs://$(gcloud config get-value project)/temp

gs://active-bolt-265320/temp


• Copie e cole o nome completo do balde e pasta acima para o campo "local temporario" (temporary location)<br/>
• Clique em "Executar Job" (run job)

<h2>Tarefa 6: Determine as taxas de throughput</h2>
Uma atividade comum ao monitorar e aprimorar os pipelines do Dataflow é descobrir quantos elementos o pipeline processa por segundo, qual é o atraso do sistema e quantos elementos de dados foram processados até o momento. Nesta atividade, você aprenderá onde, no Cloud Console, é possível encontrar informações sobre elementos processados e tempo.<br/><br/>
1 - Abra o console do Dataflow e clique no primeiro job criado, que ja deve estar funcionando plenamente.<br/><br/>
2 - Clique na primeira transformação do gráfico (a caixa superior no gráfico) e analise as métricas à direita.<br/><br/>
• O <b>System Lag</b> é uma métrica importante para streaming de pipelines. Representa a quantidade de tempo que os elementos de dados estão aguardando para serem processados desde que chegaram na entrada da etapa de transformação.<br/><br/>
• a métrica <b>Elements Added</b> dentro de output collections informa quantos elementos de dados saíram desta etapa (para a etapa seguinte, e também representa o número de mensagens do Pub/Sub lidas do tópico).<br/>

<h2>Tarefa 7: Consulte os dados no BigQuery</h2>
Consulte o novo job no Dataflow. Este job também levará alguns minutos para começar a funcionar plenamente. A tabela só será preenchida quando o fluxo estiver funcionando plenamente.
<br/><br/>
Abra o console do BigQuery. Clique na tabela e escolha "consultar a tabela" (query table) e execute a query para ver 1000 linhas.<br/><br/>


<h2>Execute as tarefas 8, 10, 11, 12, 13 do Exercício original </h2>
Cuidado para selecionar os itens criados neste notebook, e não os citados originalmente no exercício.